In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
import LoadDataset as LD
import numpy as np
import scipy.misc
from scipy.misc import imresize

Using TensorFlow backend.


In [2]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 5

In [3]:
n_categories = 3
categories = np.array(["c0", "c1"])

load_data = LD.LoadData('/netscratch/data/state_farm/imgs/train') # give the directory address.
X ,Y = load_data.generate_data()
X, Y = load_data.shuffle_data(X,Y)

FileNotFoundError: [Errno 2] No such file or directory: '/netscratch/data/state_farm/imgs/train'

In [10]:
X = np.array(X)
Y = np.array(Y)
X.shape

(22424, 480, 640, 3)

In [11]:
X_ = imresize(X, (22424, 227, 227, 3))
X_.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """Entry point for launching an IPython kernel.


ValueError: 'arr' does not have a suitable array shape for any mode.

In [5]:
(x_train, y_train) = (X[0:20000, 0:227, 0:227,], Y[0:20000])
(x_test, y_test) = (X[20001:, 0:227, 0:227,], Y[20001:])
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

227 227 3
(20000, 10)


In [9]:
# for cifar10 dataset
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)

img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [6]:
# AlexNet Define the Model
model = Sequential()
# model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='valid', input_shape=(img_height, img_width, channel,)))
# for original Alexnet
model.add(Conv2D(96, (3,3), strides=(2,2), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
# Local Response normalization for Original Alexnet
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
# print the model summary
model.summary()

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train the Model

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 114, 114, 96)      2688      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 57, 57, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 57, 57, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 384)       885120    
__________

In [8]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 14.634671624514667
Test accuracy: 0.09203466775493094
